In [ ]:
import os
import cv2

import open3d as o3d
import numpy as np

from tqdm import tqdm
from PIL import Image

import matplotlib.pyplot as plt


In [ ]:
PATH = '/home/weders/scratch/scratch/scannetter/arkit/raw/Validation'
SCENE = 'scene0458_00'


In [ ]:
label_name = ''

scene_path = os.path.join(PATH, SCENE)
image_path = os.path.join(scene_path, 'color')
depth_path = os.path.join(scene_path, 'depth')
intrinsics_path = os.path.join(scene_path, 'intrinsic')
pose_path = os.path.join(scene_path, 'pose')
# label_path = os.path.join(scene_path, 'label-proc')

mesh_path = os.path.join(scene_path, f'{SCENE}_vh_clean.ply')


In [ ]:
# scannet label coloring
def create_color_palette():
  return [
      (0, 0, 0),
      (174, 199, 232),  # wall
      (152, 223, 138),  # floor
      (31, 119, 180),  # cabinet
      (255, 187, 120),  # bed
      (188, 189, 34),  # chair
      (140, 86, 75),  # sofa
      (255, 152, 150),  # table
      (214, 39, 40),  # door
      (197, 176, 213),  # window
      (148, 103, 189),  # bookshelf
      (196, 156, 148),  # picture
      (23, 190, 207),  # counter
      (178, 76, 76),
      (247, 182, 210),  # desk
      (66, 188, 102),
      (219, 219, 141),  # curtain
      (140, 57, 197),
      (202, 185, 52),
      (51, 176, 203),
      (200, 54, 131),
      (92, 193, 61),
      (78, 71, 183),
      (172, 114, 82),
      (255, 127, 14),  # refrigerator
      (91, 163, 138),
      (153, 98, 156),
      (140, 153, 101),
      (158, 218, 229),  # shower curtain
      (100, 125, 154),
      (178, 127, 135),
      (120, 185, 128),
      (146, 111, 194),
      (44, 160, 44),  # toilet
      (112, 128, 144),  # sink
      (96, 207, 209),
      (227, 119, 194),  # bathtub
      (213, 92, 176),
      (94, 106, 211),
      (82, 84, 163),  # otherfurn
      (100, 85, 144)
  ]


def colorize_semantic_pointcloud(labels):
  colors = 255 * np.ones((labels.shape[0], 3))
  color_palette = np.asarray(create_color_palette())

  for l in np.unique(labels):
    colors[labels == l] = color_palette[l, :]

  return colors


# color by label
def visualize_label_image(image):
  height = image.shape[0]
  width = image.shape[1]
  vis_image = np.zeros([height, width, 3], dtype=np.uint8)
  color_palette = create_color_palette()
  for idx, color in enumerate(color_palette):
    vis_image[image == idx] = color
  return vis_image


In [ ]:
# load mesh
mesh = o3d.io.read_triangle_mesh(mesh_path)
vertices = np.asarray(mesh.vertices)
colors = np.asarray(mesh.vertex_colors)
labels_3d = np.zeros((vertices.shape[0], 2000))


In [ ]:
def project_pointcloud(points, pose, intrinsics):

  points_h = np.hstack((points, np.ones_like(points[:, 0:1])))
  points_c = np.linalg.inv(pose) @ points_h.T
  points_c = points_c.T
  points_p = intrinsics @ points_c.T
  points_p = points_p.T[:, :3]

  points_p[:, 0] /= (points_p[:, -1] + 1.e-6)
  points_p[:, 1] /= (points_p[:, -1] + 1.e-6)

  return points_p


files = [f for f in os.listdir(label_path) if f.endswith('png')]
files = sorted(files, key=lambda x: int(x.split('.')[0]))
resize_image = False
subsampling = 1

for idx, file in tqdm(enumerate(files), total=len(files)):

  if idx % 10 != 0:
    continue

  frame_key = int(file.split('.')[0]) * subsampling

  image = np.asarray(Image.open(os.path.join(
      image_path, f'{frame_key}.jpg'))).astype(np.uint8)
  depth = np.asarray(Image.open(os.path.join(
      depth_path, f'{frame_key}.png'))).astype(np.float32) / 1000.
  labels = np.asarray(Image.open(os.path.join(label_path, file)))

  if resize_image:
    h, w = depth.shape
    image = cv2.resize(image, (w, h))
    labels = cv2.resize(labels, (w, h))
  else:
    h, w, _ = image.shape
    depth = cv2.resize(depth, (w, h))

  if not intrinsics_loaded:
    intrinsics = np.loadtxt(intrinsics_path + '/intrinsic_color.txt')
    #         intrinsics = o3d.camera.PinholeCameraIntrinsic(width=w, height=h, fx=intrinsics[0, 0], fy=intrinsics[1, 1], cx=intrinsics[0, 2], cy=intrinsics[1, 2])
    intrinsics_loaded = False

  pose_file = os.path.join(pose_path, f'{frame_key}.txt')
  pose = np.loadtxt(pose_file)

  points_p = project_pointcloud(vertices, pose, intrinsics)

  xx = points_p[:, 0].astype(int)
  yy = points_p[:, 1].astype(int)
  zz = points_p[:, 2]

  valid_mask = (xx >= 0) & (yy >= 0) & (xx < w) & (yy < h)

  d = depth[yy[valid_mask], xx[valid_mask]]

  valid_mask[valid_mask] = (zz[valid_mask] > 0) & (np.abs(zz[valid_mask] - d)
                                                   <= 0.1)

  image_rendered = np.zeros_like(image)
  image_rendered[yy[valid_mask], xx[valid_mask], :] = colors[valid_mask] * 255
  print(labels.shape)
  labels_2d = labels[yy[valid_mask], xx[valid_mask]]
  labels_3d[valid_mask, labels_2d] += 1

  fig, ax = plt.subplots(1, 3)
  ax[0].imshow(image)
  ax[1].imshow(image_rendered)
  ax[2].imshow(visualize_label_image(labels))
  plt.show()


In [ ]:
labels_3d = np.argmax(labels_3d, axis=-1)
label_colors = colorize_semantic_pointcloud(labels_3d)


In [ ]:
from copy import deepcopy


In [ ]:
label_colors = label_colors.astype(np.float32) / 255.
mesh_colored = deepcopy(mesh)
mesh_colored.vertex_colors = o3d.utility.Vector3dVector(label_colors)


In [ ]:
o3d.io.write_triangle_mesh('label_mesh.ply', mesh_colored)


In [ ]:
tsdf = o3d.pipelines.integration.ScalableTSDFVolume(
    sdf_trunc=0.06,
    voxel_length=0.02,
    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

intrinsics_loaded = False

files = sorted(os.listdir(depth_path), key=lambda x: int(x.split('.')[0]))
pcds = None
resize_image = False

for idx, file in tqdm(enumerate(files), total=len(files)):

  if idx not in [136, 137]:
    continue

  if not os.path.exists(os.path.join(image_path, file.replace('.png', '.jpg'))):
    print(file, 'not found')
    continue

  image = np.asarray(
      Image.open(os.path.join(image_path,
                              file.replace('.png', '.jpg')))).astype(np.uint8)
  depth = np.asarray(Image.open(os.path.join(depth_path, file))).astype(
      np.float32) / 1000.

  if resize_image:
    h, w = depth.shape
    image = cv2.resize(image, (w, h))
  else:
    h, w, _, image.shape
    depth = cv2.resize(depth, (w, h))

  if not intrinsics_loaded:
    intrinsics = np.loadtxt(intrinsics_path + '/intrinsic_depth.txt')
    intrinsics = o3d.camera.PinholeCameraIntrinsic(width=w,
                                                   height=h,
                                                   fx=intrinsics[0, 0],
                                                   fy=intrinsics[1, 1],
                                                   cx=intrinsics[0, 2],
                                                   cy=intrinsics[1, 2])
    intrinsics_loaded = False

  pose_file = os.path.join(pose_path, file.replace('.png', '.txt'))
  pose = np.loadtxt(pose_file)

  image = o3d.geometry.Image(image)
  depth = o3d.geometry.Image(depth)

  rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
      image,
      depth,
      depth_scale=1.0,
      depth_trunc=3.,
      convert_rgb_to_intensity=False)

  if pcds is None:

    pcds = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd, intrinsics, pose).voxel_down_sample(0.04)

  else:
    pcds = pcds + o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd, intrinsics, np.linalg.inv(pose))
    pcds = pcds.voxel_down_sample(0.01)

o3d.io.write_point_cloud(f'{SCENE}.ply', pcds)
